In [10]:
from langchain import SQLDatabase

# dbname = "Chinook_Sqlite.sqlite"
dbname = "grocy.db"
db = SQLDatabase.from_uri(f"sqlite:///{dbname}")


c:\Users\smarotta\PycharmProjects\grocy_custom_api\venv\lib\site-packages\langchain_community\utilities\sql_database.py:122: SAWarning: Skipped unsupported reflection of expression-based index ix_products_performance2
  self._metadata.reflect(


In [11]:
import sqlite3


con = sqlite3.connect(dbname)
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('migrations',), ('locations',), ('sqlite_sequence',), ('quantity_units',), ('chores',), ('batteries',), ('recipes',), ('users',), ('sessions',), ('api_keys',), ('chores_log',), ('task_categories',), ('product_groups',), ('user_settings',), ('equipment',), ('recipes_nestings',), ('recipes_pos',), ('stock',), ('shopping_list',), ('shopping_lists',), ('userfields',), ('quantity_unit_conversions',), ('userentities',), ('userobjects',), ('meal_plan',), ('shopping_locations',), ('product_barcodes',), ('user_permissions',), ('permission_hierarchy',), ('tasks',), ('battery_charge_cycles',), ('meal_plan_sections',), ('stock_log',), ('userfield_values',), ('products',), ('cache__quantity_unit_conversions_resolved',), ('cache__products_average_price',), ('cache__products_last_purchased',)]


In [12]:
def get_schema(db):
    schema = db.get_table_info()
    return schema


In [13]:
# Connect to the database file
conn = sqlite3.connect(dbname)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Query to get the schema information
cursor.execute("SELECT name, sql FROM sqlite_master WHERE type='table'")

# Fetch the results of the query
tables = cursor.fetchall()

# Iterate over the tables and print the schema information
interesting_tables = ["locations", "quantity_units", "chores", "product_groups", "stock", "stock_log", "shopping_list", "shopping_lists", "products"]
schema = ""
for table in tables:
    table_name = table[0]
    table_schema = table[1]
    
    # if table_name in interesting_tables:
    schema += table_schema + "\n\n"

print(f"Table Schema:\n{schema}\n")
# Close the cursor and the connection
cursor.close()
conn.close()

Table Schema:
CREATE TABLE migrations (migration INTEGER NOT NULL PRIMARY KEY UNIQUE, execution_time_timestamp DATETIME DEFAULT (datetime('now', 'localtime')))

CREATE TABLE locations (
	id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
	name TEXT NOT NULL UNIQUE,
	description TEXT,
	row_created_timestamp DATETIME DEFAULT (datetime('now', 'localtime'))
, is_freezer TINYINT NOT NULL DEFAULT 0, active TINYINT NOT NULL DEFAULT 1 CHECK(active IN (0, 1)))

CREATE TABLE sqlite_sequence(name,seq)

CREATE TABLE quantity_units (
	id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
	name TEXT NOT NULL UNIQUE,
	description TEXT,
	row_created_timestamp DATETIME DEFAULT (datetime('now', 'localtime'))
, name_plural TEXT, plural_forms TEXT, active TINYINT NOT NULL DEFAULT 1 CHECK(active IN (0, 1)))

CREATE TABLE "chores" (
	id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
	name TEXT NOT NULL UNIQUE,
	description TEXT,
	period_type TEXT NOT NULL,
	period_days INTEGER,
	row_created_timestamp

In [14]:
import json


conn = sqlite3.connect('grocy.db')
cursor = conn.cursor()

query = """
    SELECT p.name AS product_name, 
           l.name AS location_name, 
           s.amount AS product_quantity
    FROM products AS p
    JOIN stock AS s ON p.id = s.product_id
    JOIN locations AS l ON p.location_id = l.id;
"""

cursor.execute(query)

# Fetch all the rows
rows = cursor.fetchall()

# Close the cursor and connection
cursor.close()
conn.close()

data = []
for row in rows:
    data.append({
        "product_name": row[0],
        "location_name": row[1],
        "product_quantity": row[2]
    })


# Write the data to a JSON file
with open('output.json', 'w') as f:
    json.dump(data, f, indent=4)

TypeError: unhashable type: 'list'

In [1]:
from langchain_community.document_loaders import JSONLoader
import json
from pathlib import Path
from pprint import pprint


file_path='output.json'
data = json.loads(Path(file_path).read_text())

In [15]:
from langchain.docstore.document import Document

documents = [Document(page_content=x["product_name"], metadata={key: value for key, value in x.items() if key != "product_name"}) for x in data]


In [18]:
documents

[Document(page_content='felpa blu quechua cappuccio', metadata={'location_name': 'Armadio', 'product_quantity': 1}),
 Document(page_content='felpa grigia quechua', metadata={'location_name': 'Armadio', 'product_quantity': 1}),
 Document(page_content='giubbotto verde', metadata={'location_name': 'Armadio', 'product_quantity': 1}),
 Document(page_content='felpa verde scout', metadata={'location_name': 'Armadio', 'product_quantity': 1}),
 Document(page_content='maglioncino azzurro', metadata={'location_name': 'Armadio', 'product_quantity': 1}),
 Document(page_content='camicia blu scuro', metadata={'location_name': 'Armadio', 'product_quantity': 1}),
 Document(page_content='maglione blu scuro cappuccio', metadata={'location_name': 'Armadio', 'product_quantity': 1}),
 Document(page_content='felpa beige pecora', metadata={'location_name': 'Armadio', 'product_quantity': 1}),
 Document(page_content='camicia blu scuro Anthony morato', metadata={'location_name': 'Armadio', 'product_quantity': 1}

In [17]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

db = Chroma.from_documents(documents, OpenAIEmbeddings())

In [45]:
query = "felpa"
docs = db.similarity_search(query)
# for d in docs:
#     print("product name", d.page_content)
#     for m in d.metadata.items():
#         print(f"{m[0].replace('_', ' ')}: {m[1]}")
#     print("\n")


formatted_docs = ""
for d in docs:
    formatted_docs += f"\n\nproduct name: {d.page_content}\n"
    for m in d.metadata.items():
        formatted_docs += f"{m[0].replace('_', ' ')}: {m[1]}\n"
print(formatted_docs)



product name: felpa beige pecora
location name: Armadio
product quantity: 1


product name: felpa verde scout
location name: Armadio
product quantity: 1


product name: felpa grigia quechua
location name: Armadio
product quantity: 1


product name: felpa blu quechua cappuccio
location name: Armadio
product quantity: 1



In [46]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Rispondi alla domanda basandoti sul seguente contesto:

{context}

Domanda: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()


def format_docs(docs):
    formatted_docs = ""
    for d in docs:
        formatted_docs += f"\n\nproduct name: {d.page_content}\n"
        for m in d.metadata.items():
            formatted_docs += f"{m[0].replace('_', ' ')}: {m[1]}\n"

    return formatted_docs

retriever = db.as_retriever()
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("Dove ho conservato il portagli?")




product name: portafogli marrone
location name: 1 cassetto esterno
product quantity: 1


product name: passaporto
location name: 1 cassetto esterno
product quantity: 1


product name: maglioncino azzurro
location name: Armadio
product quantity: 1


product name: maglione nero cerniera
location name: Armadio
product quantity: 1



'Hai conservato il portafogli nel 1 cassetto esterno.'

In [49]:
q = "Dove ho messo lo scatolo coi tappi delle cuffie?"
format_docs(db.similarity_search(q))



product name: scatolo tappi cuffie
location name: Anta inferiore dx
product quantity: 1


product name: scatolo cuffie anker
location name: Anta inferiore dx
product quantity: 1


product name: scatolo cavi
location name: Anta inferiore dx
product quantity: 1


product name: cuffie wiereless OnePlus
location name: 3 cassetto esterno
product quantity: 1



'\n\nproduct name: scatolo tappi cuffie\nlocation name: Anta inferiore dx\nproduct quantity: 1\n\n\nproduct name: scatolo cuffie anker\nlocation name: Anta inferiore dx\nproduct quantity: 1\n\n\nproduct name: scatolo cavi\nlocation name: Anta inferiore dx\nproduct quantity: 1\n\n\nproduct name: cuffie wiereless OnePlus\nlocation name: 3 cassetto esterno\nproduct quantity: 1\n'